## Notebook for using hidden Markov chains to separate signal from noise

### Purpose - To find segments of elevated signals in fluorescence recordings 
In recordings of fluorescent calcium indicators we obtain a signal whose dynamics is influenced by neuronal spiking, a variety of $Ca^{2+}$ channels and pumps, and intrinsic cellular mechanisms that involve changes in $Ca^{2+}$ concentration.
Accordingly, the signals we record can take a variety of dynamic properties and time scales. To find signals in noisy recordings, we require a method that doesn't make strong assumptions about the dynamics.

### Approach - Signal and noise have time-separable statistic properties 
A time series $X(t)$ is considered as a realization of a hidden Markov model (HMM), defined by a sequence of **states**, $S(t)$, and **emission** functions, $F_s(x;\Theta_s)$. 
In this case we choose the emission functions to be Gaussians with parameters $\Theta_s = (\mu_s,\sigma_s)$ and allow only 2 states (noise and signal). 
The fitting algorithm estimates these parameters and the transition probability matrix, $A_{ss'}$ that determines the memoriless process that governs the HMM and optimizes the loglikelihood: $$L(X;\Theta,A) = \sum_{t}\log(A_{s(t-1),s(t)}F_{s(t)}(x(t);\Theta_{s(t)}))$$
Then, the fitted model is used to estimate the MAP sequence of states, $S^*(t)$, given the observation $X$.

#### Assumptions
* Gaussian emission functions - If signals have multiple sources then this is a reasonable assumption (following the central limit theorem). If signals don't follow a Gaussian distribution and the fit shows a systematic failure then a different distribution can be tested. (e.g. a mixture of Gaussians)
* Markov chain and sparseness assumptions - The memoriless markov chain, represented by the transition probability matrix, $A$, is equal to the assumption that the signal epochs follow an exponential distribution. In the code below, we also assume that the signal is sparse and assign signal epochs with the less frequent state. 

### Data structures

#### Input files:
The required input is one .mat file per trial that contain the variable **dff** [size = n_rois x n_time_bins]. This notebook requires a base directory (set the parameter **base_dir** below) and assumes that subfolders contain the data.

#### Output files:
If files are processed separately then each output file has the same name as the corresponding input file but with the prefix **Events_**. Each file contains the variable **map_states** (The max. a-posteriori estimate of the states) that has the same dimensions as **dff**. 

If the files are concatenated then the ouput is in the file **AllEvents.mat** and it contains also the series of concatenated lengths in the variable **lengths**.

### Data concatenation
It's possible to either concatenate the time series for each ROI for all files from each day and then run the estimation or run the estimation for each file separately. To concatenate first, set the parameter **to_concatenate** to 1.



In [5]:
# imports
import os
import glob
import scipy.io as cpio
import numpy as np
import time
from hmmlearn import hmm
import matplotlib.pyplot as plt
%matplotlib inline

In [72]:
# Parameters
to_concatenate = 0
frames_to_ignore = 0
# working directory
base_dir = '/Users/yardenc/Documents/Experiments/Imaging/Data/CanaryData/lrb853_15/ManualROIs/ROIdata/'
os.chdir(base_dir)

In [73]:
dir_list = glob.glob('*/')
# Assume all subdirectories contain only data files that start with the prefix 'ROIdata'
for dirnum in range(len(dir_list)):
    os.chdir(base_dir + dir_list[dirnum])
    file_list = glob.glob('ROIdata*.mat')
    if not len(file_list) == 0:  
        for filenum in range(len(file_list)):
            print('Loading: ' + file_list[filenum])
            data = cpio.loadmat(file_list[filenum])
            dff = np.array(data['dff'])
            if to_concatenate == 1:
                if filenum > 0:
                    tot_dff = np.concatenate((tot_dff,dff),axis=1)
                    lengths = np.concatenate([lengths, len(dff) - frames_to_ignore])
                else:
                    tot_dff = dff
                    lengths = np.array(len(dff) - frames_to_ignore)   
            else:
                for roin in range(len(dff)):
                    # Here starts the code that actually estimates the states sequence for the input X
                    X = dff[roin][frames_to_ignore:].reshape(-1,1)
                    X = np.array([x if not np.isnan(x) else np.random.normal()*0.0001 for x in X])
                    X = X - np.mean(X)
                    X = X.reshape(-1,1)
                    model = hmm.GaussianHMM(n_components=2, covariance_type="full", n_iter=100)
                    model.fit(X) 
                    map_states = model.predict(X)
                    map_states = np.abs(np.median(map_states)-map_states)
                    # Here the estimation is finished
                    # Now we make sure that events with very small or negative values will not be considered
                    # The limit on the maximal value is set to 0.02 and it is not met then the segment is considered 
                    # as noise
                    edges = np.diff(np.concatenate((np.zeros(1),map_states,np.zeros(1))))
                    on_edges = [a for a,b in enumerate(edges) if b == 1]
                    off_edges = [a for a,b in enumerate(edges) if b == -1]
                    for cnt in range(len(on_edges)):
                        if np.max(X[on_edges[cnt]:off_edges[cnt]]) < 0.02:
                            map_states[on_edges[cnt]:off_edges[cnt]] = 0
                    if roin == 0:
                        signal = np.concatenate((np.zeros(frames_to_ignore),map_states))
                    else:
                        signal = np.vstack((signal,np.concatenate((np.zeros(frames_to_ignore),map_states))))
                    
            outname = 'Events_'+ file_list[filenum]
            print('Saving: ' + outname)
            cpio.savemat(outname ,{'map_states':signal})
        if to_concatenate == 1:
            for roin in range(len(dff)):
                    X = tot_dff[roin].reshape(-1,1)
                    X = X - np.mean(X)
                    model = hmm.GaussianHMM(n_components=2, covariance_type="full", n_iter=100)
                    model.fit(X,lengths) 
                    map_states = model.predict(X)
                    map_states = np.abs(np.median(map_states)-map_states)
                    edges = np.diff(np.concatenate((np.zeros(1),map_states,np.zeros(1))))
                    on_edges = [a for a,b in enumerate(edges) if b == 1]
                    off_edges = [a for a,b in enumerate(edges) if b == -1]
                    for cnt in range(len(on_edges)):
                        if np.max(X[on_edges[cnt]:off_edges[cnt]]) < 0.02:
                            map_states[on_edges[cnt]:off_edges[cnt]] = 0
                    if roin == 0:
                        signal = np.concatenate((np.zeros(frames_to_ignore),map_states))
                    else:
                        signal = np.vstack((signal,np.concatenate((np.zeros(frames_to_ignore),map_states))))
            outname = ['AllEvents.mat']
            cpio.savemat(outname ,{'map_states':signal, 'lengths':lengths, 'frames_to_ignore':frames_to_ignore})
            
            

Loading: ROIdata_lrb85315_8124_2017_07_07_08_24_31.mat
Saving: Events_ROIdata_lrb85315_8124_2017_07_07_08_24_31.mat
Loading: ROIdata_lrb85315_8104_2017_07_06_05_37_56.mat
Saving: Events_ROIdata_lrb85315_8104_2017_07_06_05_37_56.mat
Loading: ROIdata_lrb85315_8117_2017_07_06_07_12_42.mat
Saving: Events_ROIdata_lrb85315_8117_2017_07_06_07_12_42.mat
Loading: ROIdata_lrb85315_8105_2017_07_06_05_38_57.mat
Saving: Events_ROIdata_lrb85315_8105_2017_07_06_05_38_57.mat
Loading: ROIdata_lrb85315_8108_2017_07_06_05_53_28.mat
Saving: Events_ROIdata_lrb85315_8108_2017_07_06_05_53_28.mat
Loading: ROIdata_lrb85315_8113_2017_07_06_07_07_12.mat
Saving: Events_ROIdata_lrb85315_8113_2017_07_06_07_07_12.mat
Loading: ROIdata_lrb85315_8116_2017_07_06_07_11_31.mat
Saving: Events_ROIdata_lrb85315_8116_2017_07_06_07_11_31.mat
Loading: ROIdata_lrb85315_8106_2017_07_06_05_39_16.mat
Saving: Events_ROIdata_lrb85315_8106_2017_07_06_05_39_16.mat
Loading: ROIdata_lrb85315_8107_2017_07_06_05_40_01.mat
Saving: Events_RO